In [2]:
import pandas as pd
import os, re
import numpy as np
import soundfile as sf
from IPython.display import clear_output
import pickle
import librosa
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import det_curve, DetCurveDisplay
import plotly.express as px
import plotly
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Przygotowanie danych do modelu UBM, jego trening oraz liczenie LDA.

In [ ]:
df = pd.read_csv('/kaggle/input/all-people-df/df')

# Ścieżka do folderu, w którym znajdują się katalogi z nagraniami osób.
file_path = '/kaggle/input/audio-to-train-model/train-clean-100'

# Wyciągam wszystkie nazwy podfolderów z powyższej ścieżki (są to ID nagranych osób).
subfolders = [f.name for f in os.scandir(file_path) if f.is_dir()]

# Sortuję ID nagranych osób (najpierw muszę zamienić ID na liczbę).
subfolders = sorted([int(item) for item in subfolders])
subfolders = np.array(subfolders)

df = df.loc[np.isin(np.array(df['ID']), subfolders)]

# Tworzę oddzielne ramki dla kobiet i mężczyzn.
df_woman = df[df['SEX'] == ' F '].reset_index(drop=True)
df_man = df[df['SEX'] == ' M '].reset_index(drop=True)

In [ ]:
folders_path = '/kaggle/input/audio-to-train-model/train-clean-100'

# Wyciągam wszystkie podfoldery (ID osób) z głównego folderu z nagraniami.
subfolders = [f.name for f in os.scandir(folders_path) if f.is_dir()]
# Sortuję ID osób i zamieniam je z powrotem na stringi (ID muszą być w formie tekstowej).
subfolders = sorted([int(item) for item in subfolders])
subfolders = [str(item) for item in subfolders]

# Tworzę pełne ścieżki do folderów dla każdego ID.
paths_with_ID = [folders_path + '/' + subfolder for subfolder in subfolders]

# Tworzę dwie ramki danych do przechowywania ID i sumarycznej długości nagrań dla kobiet i mężczyzn.
data_frame_for_duration_woman = pd.DataFrame(columns=['ID', 'duration'])
data_frame_for_duration_man = pd.DataFrame(columns=['ID', 'duration'])

# Pętla przez wszystkie osoby, aby obliczyć sumaryczną długość nagrań.
for path_with_ID in paths_with_ID:

    # Zbieram ścieżki do folderów wewnątrz folderu danej osoby (podfoldery).
    paths_inside_ID = [f.name for f in os.scandir(path_with_ID) if f.is_dir()]

    # Tworzę pełne ścieżki do plików nagrań (plików .flac) dla każdego folderu wewnątrz ID.
    full_paths_to_files = [path_with_ID + '/' + path_inside_ID for path_inside_ID in paths_inside_ID]

    # Zbieram wszystkie pliki audio dla danej osoby.
    all_files_for_ID = []
    for full_path_to_files in full_paths_to_files:
        files = [f.name for f in os.scandir(full_path_to_files) if f.is_file() and f.name.endswith('.flac')]
        files = [full_path_to_files + '/' + file for file in files]
        all_files_for_ID = all_files_for_ID + files

    # Obliczam łączną długość nagrań danej osoby.
    duration_in_seconds = 0
    for file_for_ID in all_files_for_ID:
        # Otwieram plik audio za pomocą SoundFile i obliczam długość nagrania na podstawie liczby próbek i częstotliwości próbkowania.
        with sf.SoundFile(file_for_ID) as f:
            frames = len(f)  # Liczba próbek (frames)
            sample_rate = f.samplerate  # Częstotliwość próbkowania
        duration = frames / sample_rate  # Długość nagrania w sekundach
        duration_in_seconds = duration_in_seconds + duration  # Sumowanie długości wszystkich nagrań

    # Wyciągam ID osoby z pełnej ścieżki.
    ID = path_with_ID.split('/')[-1]
    # Tworzę nowy rekord z ID i sumaryczną długością nagrań.
    new_record = [ID, duration_in_seconds]
    
    # Sprawdzam, czy ID osoby należy do kobiet i dodaję dane do odpowiedniej ramki danych.
    if np.isin(ID, df_woman['ID']):
        data_frame_for_duration_woman.loc[len(data_frame_for_duration_woman)] = new_record
    else:
        data_frame_for_duration_man.loc[len(data_frame_for_duration_man)] = new_record

    # Wyświetlam postęp pętli.
    print(ID)
    clear_output(wait=True)

# Sortuję ramki danych według długości nagrań w kolejności malejącej.
data_frame_for_duration_woman = data_frame_for_duration_woman.sort_values(by='duration', ascending=False)
data_frame_for_duration_man = data_frame_for_duration_man.sort_values(by='duration', ascending=False)

data_frame_for_duration_woman.to_csv('data_frame_for_duration_woman.csv', index=False)
data_frame_for_duration_man.to_csv('data_frame_for_duration_man.csv', index=False)

# Wybieram 50 kobiet i 50 mężczyzn o najdłuższej sumarycznej długości nagrań.
top_50_man = data_frame_for_duration_man.head(50)
top_50_woman = data_frame_for_duration_woman.head(50)


In [ ]:
folders_path = '/kaggle/input/audio-to-train-model/train-clean-100'

top_50_man_paths = [folders_path + '/' + ID for ID in top_50_man['ID']]
top_50_woman_paths = [folders_path + '/' + ID for ID in top_50_woman['ID']]

top_50_man_and_woman = top_50_man_paths + top_50_woman_paths

with open("top_50_man_and_woman.pkl", "wb") as file:
    pickle.dump(top_50_man_and_woman, file)

In [ ]:
def split_audio_to_slices(path_to_files, seconds):
    
    # Przechodzę do katalogów wewnątrz folderu osoby (ID osoby).
    # Każdy folder wewnętrzny zawiera więcej podfolderów, które mogą zawierać nagrania.
    paths_inside_ID = [f.name for f in os.scandir(path_to_files) if f.is_dir()]

    # Tworzę pełne ścieżki do podfolderów, aby przejść do wszystkich plików nagrań dla danej osoby.
    full_paths_to_files = [path_to_files + '/' + path_inside_ID for path_inside_ID in paths_inside_ID]

    # Zbieram wszystkie ścieżki do plików audio danej osoby.
    # Każdy plik powinien mieć rozszerzenie `.flac`, a wszystkie pliki są przechowywane w zmiennej `all_files_for_ID`.

    all_files_for_ID = []
    
    for full_path_to_files in full_paths_to_files:
        files = [f.name for f in os.scandir(full_path_to_files) if f.is_file() and f.name.endswith('.flac')]
        files = [full_path_to_files + '/' + file for file in files]
        all_files_for_ID = all_files_for_ID + files

    # Łączę wszystkie nagrania danej osoby w jedno bardzo długie nagranie.
    # Używam częstotliwości próbkowania 16kHz (standardowe dla nagrań mowy).
    sr = 16000
    combined_signals = np.array([])

    for file_for_ID in all_files_for_ID:
        signal, sr = librosa.load(file_for_ID, sr=sr)
        combined_signals = np.concatenate([combined_signals, signal])



    # Długie nagranie dzielę na  fragmenty o podanej długości.
    # Fragmenty, które mają mniej niż zadeklarowane długości nagrania (resztki na końcu nagrania), są pomijane.
    list_for_parts = []
    len_of_combined_signals = len(combined_signals)
    step = seconds * sr  # Ustawienie skoku na 5 sekund
    
    for i in np.arange(start=0, stop=len_of_combined_signals-step, step=step):
        list_for_parts.append(combined_signals[i:i+step].tolist())

    parts = np.array(list_for_parts)

    

    # Liczba współczynników MFCC, które zostaną wyliczone dla każdego fragmentu nagrania (standardowe 13 współczynników).
    quantity_of_mel_coef = 13

    # Liczba filtrów melowych, które określają, ile "czapek" melowych zostanie użytych do przetwarzania sygnału.
    quantity_of_mel_filters = 26


    # Liczę MFCC dla którkich fragmentów nagrań
    mfcc_list = []
    
    for i in range(len(parts)):
        mfcc = librosa.feature.mfcc(y=parts[i], 
                                    sr=16000, 
                                    n_mfcc=quantity_of_mel_coef, 
                                    n_mels=quantity_of_mel_filters).T
        mfcc_list.append(mfcc)

    mfcc = np.array(mfcc_list)

    # Funkcja zwraca MFCC dla nagrań o długości jednej sekundy
    return mfcc

In [ ]:
train_mfcc_list = []
train_owner = []

# W pętli korzystam z wcześniej zdefiniowanej funkcji, która dzieli nagrania każdej z 100 osób
# (50 mężczyzn i 50 kobiet) na zestaw treningowy i testowy.

for i in range(0, 100):
    
    # Wywołuję funkcję split_audio_to_slices, aby podzielić nagrania danej osoby na zbiory treningowe i testowe.
    mfcc = split_audio_to_slices(top_50_man_and_woman[i], seconds = 1)

    # Dodaję uzyskane dane MFCC do odpowiednich list.
    train_mfcc_list.extend(mfcc)

    # Do listy train_owner i test_owner dodaję identyfikatory osób (i) tyle razy, ile jest nagrań.
    train_owner.extend([i] * len(mfcc))

    # Wyświetlam numer osoby w pętli, aby śledzić postęp.
    print(i)
    clear_output(wait=True)

In [ ]:
# Liczę skaler aby sieć dobrze się wytrenowała

commmon_df_for_train = np.concatenate(train_mfcc_list, axis=0)

scaler = StandardScaler()

scaler.fit(commmon_df_for_train)

X_train = [scaler.transform(one_audio) for one_audio in train_mfcc_list]

X_train = np.array(X_train)
train_owner = np.array(train_owner)

with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [ ]:
# Wydzielam zbiór treningowy i walidacyjny do modelu
train_size = int(np.floor(len(X_train) * 0.8))

# Losowy wybór indeksów, które zostaną użyte jako dane treningowe (80% próbek).
index_of_train = np.random.choice(np.arange(0, len(X_train)), size=train_size, replace=False)

# Reszta indeksów (20% próbek będzie zbiorem walidacyjnym)
rest_of_index = ~np.isin(np.arange(0, len(X_train)), index_of_train)

X_valid = X_train[rest_of_index]
X_train = X_train[index_of_train]

valid_owner = train_owner[rest_of_index]
train_owner = train_owner[index_of_train]

In [ ]:
# Przekształcam dane treningowe (X_train), aby miały odpowiedni kształt dla sieci CNN.
# Dodaję nowy wymiar (1) na końcu, ponieważ sieci konwolucyjne oczekują wejść o formacie 4D:
# (liczba próbek, wysokość, szerokość, liczba kanałów). Tutaj mamy 1 kanał (monofoniczne nagrania).

X_train = X_train.reshape((X_train.shape[0], 
                           X_train.shape[1], 
                           X_train.shape[2], 
                           1))

# Przekształcam dane walidacyjne (X_valid) do tego samego formatu 4D co dane treningowe.
X_valid = X_valid.reshape((X_valid.shape[0], 
                           X_valid.shape[1], 
                           X_valid.shape[2], 
                           1))



# Konwertuję etykiety dla zestawu treningowego (train_owner) na postać one-hot encoding dla klasyfikacji wieloklasowej (100 klas).
# Używam funkcji to_categorical, aby zamienić numeryczne etykiety (ID osób) na macierze o rozmiarze [100], gdzie 
# każda wartość reprezentuje prawdopodobieństwo przynależności do danej klasy.
y_train = to_categorical(train_owner, num_classes=100)

# Podobnie konwertuję etykiety dla zestawu walidacyjnego na one-hot encoding.
y_valid = to_categorical(valid_owner, num_classes=100)

In [ ]:
np.save('X_train.npy', X_train)
np.save('X_valid.npy', X_valid)

np.save('y_train.npy', y_train)
np.save('y_valid.npy', y_valid)

In [ ]:
X_train = np.load('/kaggle/input/ubm-train-data/X_train.npy')
X_valid = np.load('/kaggle/input/ubm-train-data/X_valid.npy')

y_train = np.load('/kaggle/input/ubm-train-data/y_train.npy')
y_valid = np.load('/kaggle/input/ubm-train-data/y_valid.npy')

In [ ]:
y_train_labels = np.argmax(y_train, axis=1)

In [ ]:
# Ustawienie ziarna losowości dla powtarzalnych wyników
np.random.seed(11)
tf.random.set_seed(11)
np.random.seed(11)

# Inicjalizacja modelu
model = models.Sequential()

# Dodanie warstwy wejściowej z kształtem zgodnym z danymi MFCC 
model.add(layers.Input(shape=(32, 13, 1)))

# Pierwsza warstwa konwolucyjna, która ma 64 filtry, okno o rozmiarze 3x3, ReLU jako funkcję aktywacji oraz padding 'same'
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
# Normalizacja wsadowa dla lepszego uczenia
model.add(layers.BatchNormalization())
# Max Pooling dla zmniejszenia rozmiaru przestrzennego o współczynnik 2
model.add(layers.MaxPooling2D((2, 2)))
# Dropout dla zapobiegania przeuczeniu (wyłącza 10% neuronów losowo)
model.add(layers.Dropout(0.1))

# Druga warstwa konwolucyjna, zwiększamy liczbę filtrów do 128
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.1))

# Trzecia warstwa konwolucyjna, zwiększamy liczbę filtrów do 256
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.1))

# Czwarta warstwa konwolucyjna, zwiększamy liczbę filtrów do 512
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
# Dropout dla zapobiegania przeuczeniu (tym razem wyłącza 20% neuronów)
model.add(layers.Dropout(0.2))

# Spłaszczenie wyniku do wektora jednowymiarowego przed przekazaniem go do warstw w pełni połączonych
model.add(layers.Flatten())

# Pierwsza warstwa w pełni połączona (Dense) z 2048 jednostkami, funkcja aktywacji ReLU
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
# Dropout dla zapobiegania przeuczeniu (wyłącza 75% neuronów)
model.add(layers.Dropout(0.75))

# Warstwa bottleneck
model.add(layers.Dense(64, activation='linear'))





# Warstwa wyjściowa - klasyfikacja wieloklasowa z 100 jednostkami i aktywacją softmax (klasyfikacja wieloklasowa)
model.add(layers.Dense(100, activation='softmax'))  # 100 klas (każda klasa reprezentuje innego mówcę)

# Kompilacja modelu - optymalizator Adam, funkcja straty categorical_crossentropy (wieloklasowa klasyfikacja), oraz miara skuteczności - accuracy
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Wyświetlenie podsumowania modelu
model.summary()

# Callbacks: redukcja współczynnika uczenia (learning rate), gdy val_loss przestaje się poprawiać
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-8)
# Wczesne zatrzymanie treningu, jeśli val_loss nie poprawia się przez 10 epok, przywracając najlepsze wagi
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Trenowanie modelu
history = model.fit(
    X_train,    # Dane MFCC
    y_train,    # Etykiety one-hot encoded (macierz klas mówców)
    epochs=500,  # Liczba epok treningu
    batch_size=32, # Rozmiar batcha (z dotychczasowych testów wynika, że 32 jest optymalne)
    verbose = 1,
    validation_data=(X_valid, y_valid),  # Walidacja na wcześniej zdefiniowanych danych walidacyjnych
    callbacks=[reduce_lr, early_stopping]  # Callbacki do dynamicznej regulacji uczenia i wczesnego zatrzymania
)

In [ ]:
model.save("model.h5")

In [3]:
# Funkcja służy do stworzenia embeddingu nagrania, pobiera po prostu wartości jakie są w wartowie bottlneck podczas dokonywania klasyfikacji danego nagrania
def calcuate_embedding(one_audio, model):
    
    intermediate_layer_model = Model(inputs=model.inputs,
                                 outputs=model.get_layer('dense_1').output)
    intermediate_output = intermediate_layer_model.predict(one_audio[np.newaxis, ...])
    
    return intermediate_output

In [ ]:
# Liczymy embeddingi nagrań treningowych UBM aby stworzyć LDA na ich podstawie
X_train_embedding = []

# Iteracja po każdym elemencie w X_train
for i in range(0, len(X_train)):
    # Obliczenie embeddingu dla danego nagrania w X_train przy użyciu modelu
    train_embedding = calcuate_embedding(X_train[i], model)

    # Dodanie embeddingu do listy X_train_embedding
    X_train_embedding.append(train_embedding)

    # Wyczyszczenie poprzedniego outputu i wyświetlenie postępu przetwarzania
    clear_output(wait=True)
    print(i/len(X_train))  # Wyświetla proporcjonalny postęp jako wartość od 0 do 1

# Zapisanie listy embeddingów X_train_embedding do pliku w formacie pickle
with open("X_train_embedding.pkl", "wb") as file:
    pickle.dump(X_train_embedding, file)


In [7]:
# Wczytanie embeddingów treningowych z pliku pickle
with open("C:/Users/zbugo/Desktop/praktyki_zadania/18/good_data/X_train_embedding.pkl", 'rb') as file:
    X_train_embedding = pickle.load(file)

# Wczytanie etykiet treningowych z pliku .npy
y_train = np.load("C:/Users/zbugo/Desktop/praktyki_zadania/18/good_data/y_train.npy")
# Zamiana etykiet one-hot encoded na numery klas
y_train_classes = np.argmax(y_train, axis=1)

# Przekształcenie listy embeddingów w macierz NumPy, gdzie każdy wiersz to embedding
X_train_embedding = np.vstack(X_train_embedding)

# Skalowanie danych do rozkładu o średniej 0 i odchyleniu standardowym 1
scaler = StandardScaler()
scaler.fit(X_train_embedding)
X_train_embedding = scaler.transform(X_train_embedding)

# Dopasowanie LDA (Linear Discriminant Analysis) na przeskalowanych embeddingach
lda = LDA()
lda.fit(X=X_train_embedding, y=y_train_classes);

In [9]:
with open('scaler_after_embedding.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open('lda.pkl', 'wb') as file:
    pickle.dump(lda, file)